# Examples of spatial querying

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fspatial_querying.ipynb)

In a first section, different examples will be shown on how to use existing webservices to setup spatial queries:

- Filter data within a certain community using its geographic borders
- Filter data within a geographic boundary of a feature in layer "Bekken"
- Filter data within a geographic boundary of a feature in layer "Hydrogeologische homogene zones"

In other situations, a spatial query is need to be defined by a file (e.g. an ESRI shapefile) containing information on the particular area of interest. In a second section of this notebook, the usage of file based queries is shown. 

__Note:__ To run the file based examples, the installation of the `fiona` package and/or the `geopandas` package is required. See the installation instructions.

In [1]:
%matplotlib inline
from io import BytesIO
import matplotlib.pyplot as plt

from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from pydov.search.boring import BoringSearch
from pydov.search.grondwaterfilter import GrondwaterFilterSearch

from pydov.util.location import (
    GmlFilter,
    Within,
    GeometryFilter, 
    GeopandasFilter
)

from pydov.util.owsutil import get_namespace

# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer
import fiona
from fiona.crs import from_epsg
import geopandas

# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

# convert to bytes if necessary
def to_bytes(data):
    if isinstance(data, bytes):
        return data
    elif isinstance(data, str):
        return data.encode('utf8')

## Spatial querying using web services

### Filter data within a certain community using its geographic borders

This example will use a third party WFS service to retrieve to geometry of (the boundary of) a community and subsequently use this boundary as a spatial filter to query boreholes from the DOV service.

We have to start by defining the third party WFS service that contains the featuretype (layer) that contains the geometry we're interested in:

In [2]:
gemeentegrenzen = WebFeatureService(
    'https://geo.api.vlaanderen.be/VRBG/wfs',
    version='1.1.0')

In this case we know in advance which featuretype from the WFS service we need (`VRBG:RefGem`) but are unsure which field (attribute) we can use to get the community we want.

We can get the schema (i.e. all the available fields) of a layer to find a field to query on:

In [3]:
gemeentegrenzen.get_schema('VRBG:Refgem')['properties']

{'UIDN': 'decimal',
 'OIDN': 'decimal',
 'TERRID': 'decimal',
 'NISCODE': 'string',
 'NAAM': 'string',
 'DATPUBLBS': 'date',
 'NUMAC': 'string'}

We can now build a search query to get the feature (including geometry) of the community with the `NAAM` of `Lievegem`:

In [4]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Lievegem')

By default, the feature is returned as a GML feature, which stands for `Geography Markup Language` and is a XML grammar used to express geographical features.
We change the `outputFormat` to GML3.2, since pydov's GmlFilter expects GML version 3.2.

In [5]:
gemeente_gml = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8"),
    outputFormat='text/xml; subtype=gml/3.2').read()


Next, we'll use the GML feature inside a `GmlFilter` query to find all boreholes that are located `Within` the community borders:

In [6]:
bs = BoringSearch()
df = bs.search(
    location=GmlFilter(gemeente_gml, Within),
    return_fields=('pkey_boring', 'x', 'y'))

[000/001] .


The `df` DataFrame now contains the `pkey_boring` together with the x and y coordinates:

In [7]:
df.head()

,pkey_boring,x,y
0,https://www.dov.vlaanderen.be/data/boring/2016...,92973.0,203481.0
1,https://www.dov.vlaanderen.be/data/boring/1894...,99605.0,198147.0
2,https://www.dov.vlaanderen.be/data/boring/1894...,99209.0,198139.0
3,https://www.dov.vlaanderen.be/data/boring/1894...,99103.0,198002.0
4,https://www.dov.vlaanderen.be/data/boring/1894...,98629.0,198057.0


We can show these result on the map as well:

In [8]:
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))

# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [9]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_boring'][loc]).add_to(marker_cluster)
fmap

### Filter data within a geographic boundary of a feature in layer "Bekken"

This example will use a third party WFS service to retrieve to geometry of a waterbody and subsequently use this boundary as a spatial filter to query groundwater screens from the DOV service.

We have to start by defining the third party WFS service that contains the featuretype (layer) that contains the geometry we're interested in:

In [10]:
bekkens = WebFeatureService(
    'https://geo.api.vlaanderen.be/Watersystemen/wfs',
    version='1.1.0'
)

We can list all the available featuretypes (layers) in the service if we are unsure which one to use:

In [11]:
list(bekkens.contents)

['Watersystemen:WsBekken',
 'Watersystemen:WsDeelbek',
 'Watersystemen:WsStrmgeb']

We can also get the schema (i.e. all the available fields) of a layer to find a field to query on:

In [12]:
bekkens.get_schema('Watersystemen:WsBekken')['properties']

{'OIDN': 'decimal',
 'UIDN': 'decimal',
 'BEKNR': 'short',
 'BEKNAAM': 'string',
 'STRMGEB': 'string'}

We can get all the distinct values of a specific field:

In [13]:
namespace = get_namespace(bekkens, 'Watersystemen:WsBekken')

tree = etree.fromstring(to_bytes(bekkens.getfeature('Watersystemen:WsBekken', propertyname='BEKNAAM').read()))
set((i.text for i in tree.findall('.//{%s}BEKNAAM' % namespace)))

{'Bekken van de Brugse Polders',
 'Bekken van de Gentse Kanalen',
 'Benedenscheldebekken',
 'Bovenscheldebekken',
 'Demerbekken',
 'Denderbekken',
 'Dijle- en Zennebekken',
 'IJzerbekken',
 'Leiebekken',
 'Maasbekken',
 'Netebekken'}

We can now build a search query to get the feature (including geometry) of the waterbody with the `BEKNAAM` of `Demerbekken`:

In [14]:
naam_filter = PropertyIsEqualTo(propertyname='BEKNAAM', literal='Demerbekken')

In [15]:
bekken_poly = bekkens.getfeature(
    typename='Watersystemen:WsBekken',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8"),
    outputFormat='text/xml; subtype=gml/3.2').read()

And use this GML feature inside a `GmlFilter` query to find all groundwater screens that are located `Within` the waterbody:

In [16]:
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    max_features = 100,  # Note - we limit the results here to 100 for the demo case
    location=GmlFilter(bekken_poly, Within),
    return_fields=('pkey_filter', 'x', 'y')
)

[000/001] .


In [17]:
df.head()

,pkey_filter,x,y
0,https://www.dov.vlaanderen.be/data/filter/2006...,185347.0,186221.00
1,https://www.dov.vlaanderen.be/data/filter/2003...,223063.2,172028.31
2,https://www.dov.vlaanderen.be/data/filter/1900...,186630.0,184140.00
3,https://www.dov.vlaanderen.be/data/filter/1900...,219095.0,181256.00
4,https://www.dov.vlaanderen.be/data/filter/1991...,179856.0,184474.00


We can show the result on the map:

In [18]:
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [19]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)
fmap

### Filter data within a geographic boundary of a feature in layer "HHZ"

This example will use the DOV WFS service to retrieve to geometry of a HHZ (hydrogeological homogenous area) and subsequently use this boundary as a spatial filter to query groundwater screens from the DOV service.

We can query the HHZ layer from DOV using pydov:

In [20]:
from pydov.search.generic import WfsSearch

hhz = WfsSearch('gw_varia:hhz')

Next we can get the field of the layer to find a field to query on:

In [21]:
hhz.get_fields()

{'id': {'name': 'id', 'definition': None, 'type': 'string', 'list': False, 'notnull': False, 'query': True, 'cost': 1}, 'hhz_naam': {'name': 'hhz_naam', 'definition': None, 'type': 'string', 'list': False, 'notnull': False, 'query': True, 'cost': 1}, 'opp_m2': {'name': 'opp_m2', 'definition': None, 'type': 'integer', 'list': False, 'notnull': False, 'query': True, 'cost': 1}, 'hhz': {'name': 'hhz', 'definition': None, 'type': 'string', 'list': False, 'notnull': False, 'query': True, 'cost': 1}, 'geom': {'name': 'geom', 'definition': None, 'type': 'geometry', 'list': False, 'notnull': False, 'query': False, 'cost': 1}}

We can now build a search query to get the feature where `hhz_naam` equals `Formatie van Mol`:

In [22]:
from owslib.fes2 import PropertyIsEqualTo as PropertyIsEqualTo2

naam_filter = PropertyIsEqualTo2(
    propertyname='hhz_naam', literal='Formatie van Mol')

In [23]:
hhz_poly = hhz.search(query=naam_filter, return_fields=('id', 'hhz_naam', 'geom'))
hhz_poly

[000/001] .


,id,hhz_naam,geom
0,2,Formatie van Mol,"MULTIPOLYGON (((203474.0966 223036.6511, 20424..."


And turn this result into a GeoPandas GeoDataFrame:

In [24]:
hhz_poly = geopandas.GeoDataFrame(hhz_poly, geometry='geom', crs='EPSG:31370')
hhz_poly

,id,hhz_naam,geom
0,2,Formatie van Mol,"MULTIPOLYGON (((203474.097 223036.651, 204245...."


And subsequently use this geometry in a search for groundwater screens:

In [26]:
from pydov.search.fields import GeometryReturnField

filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    max_features = 100,  # Note - we limit the results here to 100 for the demo case
    location=GeopandasFilter(hhz_poly, Within),
    return_fields=('pkey_filter', GeometryReturnField('geom', 4326))
)
df

[000/001] .


,pkey_filter,geom
0,https://www.dov.vlaanderen.be/data/filter/1984...,POINT Z (5.1758 51.2172 0)
1,https://www.dov.vlaanderen.be/data/filter/1989...,POINT Z (5.1796 51.2162 0)
2,https://www.dov.vlaanderen.be/data/filter/2005...,POINT Z (5.1231 51.3088 0)
3,https://www.dov.vlaanderen.be/data/filter/1999...,POINT Z (5.0182 51.2388 0)
4,https://www.dov.vlaanderen.be/data/filter/2012...,POINT Z (5.0096 51.2578 0)
...,...,...
95,https://www.dov.vlaanderen.be/data/filter/1984...,POINT Z (5.1781 51.2167 0)
96,https://www.dov.vlaanderen.be/data/filter/1900...,POINT Z (5.13 51.2298 0)
97,https://www.dov.vlaanderen.be/data/filter/1989...,POINT Z (5.1772 51.2147 0)
98,https://www.dov.vlaanderen.be/data/filter/2003...,POINT Z (4.9823 51.2785 0)


And plot the results on a map:

In [27]:
loclist = [[point.xy[1][0], point.xy[0][0]] for point in df.geom]

In [28]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)

# add filters
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)

# add HHZ polygon
hhz_poly_series = geopandas.GeoSeries(hhz_poly["geom"]).simplify(tolerance=0.001)
hhz_poly_json = hhz_poly_series.to_json()
hhz_poly_json = folium.GeoJson(data=hhz_poly_series, style_function=lambda x: {"fillColor": "orange"})
hhz_poly_json.add_to(fmap)

fmap.fit_bounds(fmap.get_bounds())
fmap

## File based spatial querying

### Using a vector file

Intead of using a webservice, we can also use a __file__ as input to define a spatial query. The pydov package provides the `GeometryFilter` to convert a vector file (e.g. ESRI shape file) into a GML file which can be used in the same way as the previous examples. In this case, we want all the boringen within the polygons defined in the vector file:

We have an example shape file prepared here, but this could be any other shape file as well:

In [29]:
shapefile = "../../tests/data/util/location/polygon_multiple_31370.shp"

In [30]:
df = bs.search(
    location=GeometryFilter(shapefile, Within),
    return_fields=('pkey_boring', 'x', 'y'))

[000/001] .


[Fiona](https://pypi.org/project/Fiona/) is used to do the data transformation, which is why this is an additional dependency when using the `GeometryFilter` functionality.

The resulting data points can be represented on a map:

In [31]:
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [32]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_boring'][loc]).add_to(marker_cluster)
fmap

### Interact with GeoPandas

In the previous example, the vector file as available on disk was used to create the spatial query. However, when working on geospatial vector data in Python the usage of [GeoPandas](https://geopandas.readthedocs.io/en/latest/) is convenient as it combines the data table functionalities of the [Pandas](https://pandas.pydata.org/docs/) package with spatial functionalities. GeoPandas also relies on fiona to read vector files, so using GeoPandas DataFrames can be used to define spatial filters as well.

In [33]:
import geopandas as gpd

shapefile = "../../tests/data/util/location/polygon_multiple_31370.shp"

# User reads geometry-file from disk (shapefile, geojson,...)
gdf = gpd.read_file(shapefile)
gdf["name"] = ["site 1", "site 2"]  # add dummy names to each of the polygones in the vector file for the demo case
gdf

,gml_id,geometry,name
0,polygon_multiple_31370.0,"POLYGON ((108636.15 194960.844, 109195.574 195...",site 1
1,polygon_multiple_31370.1,"POLYGON ((107485.786 196741.544, 108297.344 19...",site 2


Using the GeoDataFrame directly to define a spatial query:

In [34]:
my_filter = GeopandasFilter(gdf, Within)

bs = BoringSearch()
df = bs.search(
    location=my_filter,
    return_fields=('pkey_boring', 'x', 'y'))

[000/001] .


In [35]:
df

,pkey_boring,x,y
0,https://www.dov.vlaanderen.be/data/boring/2018...,108025.00,196593.00
1,https://www.dov.vlaanderen.be/data/boring/2019...,107947.29,196640.52
2,https://www.dov.vlaanderen.be/data/boring/2020...,107991.00,196706.00
3,https://www.dov.vlaanderen.be/data/boring/2022...,107842.53,196371.08
4,https://www.dov.vlaanderen.be/data/boring/2023...,108144.95,196771.09
5,https://www.dov.vlaanderen.be/data/boring/2022...,108131.66,196779.43
6,https://www.dov.vlaanderen.be/data/boring/1893...,108900.00,194425.00
7,https://www.dov.vlaanderen.be/data/boring/1894...,107618.00,196709.00
8,https://www.dov.vlaanderen.be/data/boring/1894...,107791.00,196516.00
9,https://www.dov.vlaanderen.be/data/boring/1895...,109050.00,194990.00


A user can do all sort of spatial and non-spatial operations in Geopandas. For example, one is only interested in the Boreholes of _site 1_ and selects this polygon only:

Using the `GeopandasFilter`, we can use this to create a spatial query:

In [36]:
gdf_subselection = gdf[gdf["name"] == "site 1"]

As long as the result is a GeoDataFrame with a crs defined, it can be used as input to define a spatial filter:

In [37]:
my_filter = GeopandasFilter(gdf_subselection, Within)

bs = BoringSearch()
df = bs.search(
    location=my_filter,
    return_fields=('pkey_boring', 'x', 'y'))

[000/001] .


The returned set of boreholes is now limited to Site 1:

In [38]:
df

,pkey_boring,x,y
0,https://www.dov.vlaanderen.be/data/boring/1893...,108900.0,194425.0
1,https://www.dov.vlaanderen.be/data/boring/1895...,109050.0,194990.0
2,https://www.dov.vlaanderen.be/data/boring/1895...,108742.0,194936.0


__Note:__ To select a single row (i.e. feature) of a GeoDataFrame, make sure the result is still a valid GeoDataFrame:

In [39]:
gdf.iloc[[0]]

,gml_id,geometry,name
0,polygon_multiple_31370.0,"POLYGON ((108636.15 194960.844, 109195.574 195...",site 1


instead of:

In [40]:
gdf.iloc[0]

gml_id                               polygon_multiple_31370.0
geometry    POLYGON ((108636.150020818 194960.844295764, 1...
name                                                   site 1
Name: 0, dtype: object

The latter won't be a valid input to define a spatial query.